In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os


In [3]:
os.chdir(r"C:\Users\sam\CODEs\MedLegit")  #add your own directory
data_dir = "data/"
dfs = []
for dirname, _, filenames in os.walk(data_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data/Test-1542969243754.csv
data/Test_Beneficiarydata-1542969243754.csv
data/Test_Inpatientdata-1542969243754.csv
data/Test_Outpatientdata-1542969243754.csv
data/Train-1542865627584.csv
data/Train_Beneficiarydata-1542865627584.csv
data/Train_Inpatientdata-1542865627584.csv
data/Train_Outpatientdata-1542865627584.csv


In [4]:
train_ben_df= pd.read_csv(r"data\Train_Beneficiarydata-1542865627584.csv")
train_ip_df = pd.read_csv(r"data\Train_Inpatientdata-1542865627584.csv")
train_op_df = pd.read_csv(r"data\Train_Outpatientdata-1542865627584.csv")
train_df    = pd.read_csv(r"data\Train-1542865627584.csv")

In [7]:
train_ip_df['Type']='Inpatient'
train_op_df['Type']='Outpatient'
claims_df = pd.concat([train_ip_df, train_op_df], axis=0)
claims_with_bene=claims_df.merge(train_ben_df,on='BeneID', how='left')
full_df=claims_with_bene.merge(train_df, on='Provider', how='left')
print(f"Shape after merge: {full_df.shape}")
full_df.head()



Shape after merge: (558211, 56)


,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,1,1,2,1,1,36000,3204,60,70,Yes
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,1,1,2,1,1,36000,3204,60,70,No
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,1,1,2,1,1,36000,3204,60,70,No
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,1,2,2,1,1,5000,1068,250,320,No
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,...,2,1,2,2,2,21260,2136,120,100,No


In [14]:
print(full_df.columns)


Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'AdmissionDt', 'ClmAdmitDiagnosisCode',
       'DeductibleAmtPaid', 'DischargeDt', 'DiagnosisGroupCode',
       'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',
       'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
       'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9',
       'ClmDiagnosisCode_10', 'ClmProcedureCode_1', 'ClmProcedureCode_2',
       'ClmProcedureCode_3', 'ClmProcedureCode_4', 'ClmProcedureCode_5',
       'ClmProcedureCode_6', 'Type', 'DOB', 'DOD', 'Gender', 'Race',
       'RenalDiseaseIndicator', 'State', 'County', 'NoOfMonths_PartACov',
       'NoOfMonths_PartBCov', 'ChronicCond_Alzheimer',
       'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
       'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary',
       'ChronicCond_Depression', 'Ch

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

# 1. Copy dataset
df = full_df.copy()

# 2. Target variable
y = df["PotentialFraud"].map({"No": 0, "Yes": 1})
X = df.drop("PotentialFraud", axis=1)

# 3. Handle dates (convert to duration/age)
X["ClaimStartDt"] = pd.to_datetime(X["ClaimStartDt"], errors="coerce")
X["ClaimEndDt"] = pd.to_datetime(X["ClaimEndDt"], errors="coerce")
X["DOB"] = pd.to_datetime(X["DOB"], errors="coerce")
X["DOD"] = pd.to_datetime(X["DOD"], errors="coerce")

X["ClaimDuration"] = (X["ClaimEndDt"] - X["ClaimStartDt"]).dt.days
X["Age"] = (X["ClaimStartDt"] - X["DOB"]).dt.days // 365
X["IsDead"] = X["DOD"].notna().astype(int)

# Drop raw date columns (we keep engineered ones)
X = X.drop(["ClaimStartDt","ClaimEndDt","DOB","DOD","AdmissionDt","DischargeDt"], axis=1)

# 4. Encode categorical features
cat_cols = X.select_dtypes(include=["object"]).columns
le = LabelEncoder()
for col in cat_cols:
    X[col] = X[col].fillna("Unknown")
    X[col] = le.fit_transform(X[col])

# 5. Handle NaNs in numeric columns
X = X.fillna(0)

# 6. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

# 7. SMOTE
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)

# 8. Train model
model = XGBClassifier(
    scale_pos_weight=len(y_res[y_res==0]) / len(y_res[y_res==1]),
    eval_metric="logloss",
    use_label_encoder=False
)
model.fit(X_res, y_res)

# 9. Eval
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, model.predict_proba(X_test)[:,1]))


ValueError: Input y contains NaN.